In [1]:
import tenseal as ts

In [2]:
public_context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
print("Is the context private?", ("Yes" if public_context.is_private() else "No"))
print("Is the context public?", ("Yes" if public_context.is_public() else "No"))

Is the context private? Yes
Is the context public? No


In [3]:
sk = public_context.secret_key()
print(sk)

In [4]:
# the context will drop the secret-key at this point
public_context.make_context_public()
print("Secret-key dropped")
print("Is the context private?", ("Yes" if public_context.is_private() else "No"))
print("Is the context public?", ("Yes" if public_context.is_public() else "No"))

Secret-key dropped
Is the context private? No
Is the context public? Yes


In [5]:
import tenseal as ts

context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)

In [6]:
#创建加密vector
plain_vector = [60, 66, 73, 81, 90]
encrypted_vector = ts.bfv_vector(context, plain_vector)
print("We just encrypted our plaintext vector of size:", encrypted_vector.size())
print(encrypted_vector)

We just encrypted our plaintext vector of size: 5


In [3]:
import tenseal as ts

context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=4096, plain_modulus=1032193)
context.global_scale = 2 ** 20

In [4]:
plain_vector = [60, 66, 73, 81, 90]
encrypted_vector = ts.ckks_vector(context, plain_vector)
print(encrypted_vector.size())
print(encrypted_vector)

5


In [1]:
import tenseal as ts
import numpy as np

In [2]:
def gencontext():
    context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[22 ,21, 21, 21, 21, 21, 21, 21, 21, 21])
    context.global_scale = pow(2, 21)
    context.generate_galois_keys()
    return context

In [3]:
def encrypt(context, np_tensor):
    return ts.ckks_tensor(context, np_tensor)

In [4]:
def decrypt(enc_tensor):
    return np.array(enc_tensor.decrypt().tolist())

In [5]:
def bootstrap(context, tensor):
    # To refresh a tensor with exhausted depth. 
    # Here, bootstrap = enc(dec())
    tmp = decrypt(tensor)
    return encrypt(context, tmp)

In [6]:
a = np.array([[1.,2.,3.,4.], [1.,2.,5.,4.]])
context = gencontext()
enc_a = encrypt(context, a)
enc_at = encrypt(context, a.T)
enc_b = encrypt(context, a)
res = enc_a + enc_b
# res = enc_a - enc_b
# res = enc_a * enc_b
# res = enc_a @ enc_at
print(decrypt(res))

[[2.00001927 3.99936407 6.00080306 7.99988295]
 [2.00043567 4.00134972 9.99936724 7.99948168]]


In [1]:
import tenseal as ts

In [2]:
# Setup TenSEAL context
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
          )
context.generate_galois_keys()
context.global_scale = 2**40

In [3]:
v1 = [0, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 0]

In [4]:
# encrypted vectors【编码和加密】
enc_v1 = ts.ckks_vector(context, v1)  
enc_v2 = ts.ckks_vector(context, v2)

In [12]:
# 密文+密文
result = enc_v1 + enc_v2
result.decrypt() # ~ [4, 4, 4, 4, 4]

[3.9999999989590425,
 4.000000000494298,
 4.000000002252071,
 4.000000001254584,
 3.999999999585722]

In [13]:
# 点积：<密文,密文>
result = enc_v1.dot(enc_v2)
print(result.decrypt()) # ~ [10]

[10.00000115580431]


In [14]:
matrix = [
  [73, 0.5, 8],
  [81, -5, 66],
  [-100, -78, -2],
  [0, 9, 17],
  [69, 11 , 10],
]

In [15]:
# 密文向量*明文矩阵
result = enc_v1.matmul(matrix)
print(result.decrypt()) # ~ [157, -90, 153]

[157.00002083897044, -90.0000121386421, 153.00002058846434]


In [1]:
# make TenSEALContext object which stores keys and parameters
import tenseal as ts

context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
          )
context.generate_galois_keys()
context.global_scale = 2**40

# coeff_mod_bit_sizes: 系数模数大小，[60, 40, 40, 60]
# global_scale: 缩放因子